# Logiques Avancées - DL, Modale, QBF, Conditional

**Navigation**: [← Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-4-Belief-Revision →](Tweety-4-Belief-Revision.ipynb)

---

## Objectifs pédagogiques

1. Comprendre les logiques de description (DL) et leur lien avec les ontologies
2. Explorer la logique modale avec les opérateurs de nécessité et possibilité
3. Découvrir la logique QBF (Quantified Boolean Formulas)
4. Appréhender la logique conditionnelle (CL)

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Limitations connues (Tweety 1.28):**
> - `SimpleMlReasoner` peut bloquer indéfiniment sans solveur SPASS configuré
> - Le parsing de logique modale fonctionne, mais le raisonnement nécessite SPASS
> - QBF et CL sont présentés en aperçu (fonctionnalités de base)

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE, pas l'executable
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover) - executable complet
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
        else:
            print(f"  {tool}: non configure")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JDK portable: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe

JVM prete. Outils: 3/3


### Interpretation de l'initialisation

**Sortie obtenue:**
- JVM demarrée avec 35 JARs Tweety
- JDK portable Zulu 17 détecté automatiquement
- 3/3 outils externes configurés: CLINGO, SPASS, EPROVER

**Signification:**
- L'environnement est prêt pour tous les exemples de ce notebook
- SPASS (prouveur modal) est disponible, mais peut nécessiter des privilèges admin sous Windows
- CLINGO sera utilisé pour la logique ASP (notebook 6)
- EProver peut servir d'alternative pour la logique du premier ordre

> **Note:** Si l'un des outils n'est pas détecté, les exemples correspondants afficheront un avertissement mais le notebook restera exécutable.

### 2.3 Logique de Description (DL)
<a id="2.3"></a>

Les logiques de description sont une famille de formalismes pour représenter des connaissances structurées, souvent utilisées pour les ontologies (ex: OWL). Elles se concentrent sur la définition de **Concepts** (classes d'individus), de **Rôles** (relations binaires) et d'**Individus**.

* **Signature (`DlSignature`)** : Comprend `AtomicConcept`, `AtomicRole`, `Individual`.
* **Axiomes** :
    * **TBox (Terminological Box)** : Axiomes définissant les concepts et les rôles (ex: `SubConceptAxiom`, `EquivalenceAxiom`, `DisjointAxiom`). Les concepts peuvent être combinés (`Intersection`, `Union`, `Complement`, `ExistsRestriction`, `ForAllRestriction`).
    * **ABox (Assertional Box)** : Axiomes sur les individus (ex: `ConceptAssertion` - `Human(Alice)`, `RoleAssertion` - `fatherOf(Bob, Alice)`).
* **Base de Connaissances (`DlBeliefSet`)** : Contient les axiomes TBox et ABox.
* **Raisonnement (`DlReasoner`)** : Vérifie la consistance, la subsomption de concepts, l'instanciation. `NaiveDlReasoner` est une implémentation simple. Des raisonneurs plus puissants (comme Pellet, HermiT - non intégrés directement comme solveurs externes dans cet exemple) existent.

In [2]:
# --- 2.3.1 Imports DL ---
print("--- 2.3.1 Logique de Description : Imports ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree. Impossible de continuer cet exemple.")
    dl_imports_ok = False
else:
    print("JVM prete. Import des classes DL...")
    dl_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        from java.util import ArrayList

        from org.tweetyproject.logics.dl.syntax import (
            AtomicConcept, AtomicRole, Individual,
            EquivalenceAxiom, ConceptAssertion, RoleAssertion,
            DlBeliefSet, DlSignature, DlAxiom,
            Complement, Union
        )
        from org.tweetyproject.logics.dl.parser import DlParser
        from org.tweetyproject.logics.dl.reasoner import NaiveDlReasoner

        print("Imports DL reussis.")
        dl_imports_ok = True

    except ImportError as e:
        print(f"ERREUR d'import pour DL : {e}")
    except Exception as e_gen:
        print(f"ERREUR inattendue : {e_gen}")

--- 2.3.1 Logique de Description : Imports ---
JVM prete. Import des classes DL...
Imports DL reussis.


### Architecture de la Logique de Description

La logique de description (DL) sépare les connaissances en deux niveaux:

| Composant | Rôle | Exemple |
|-----------|------|---------|
| **TBox** (Terminologie) | Définit les concepts et leurs relations | `Male ⊑ Human`, `Female ≡ ¬Male` |
| **ABox** (Assertions) | Faits sur les individus concrets | `Alice : Female`, `fatherOf(Bob, Alice)` |

**Opérateurs DL:**
- `⊓` (Intersection): `Male ⊓ Human` - homme ET humain
- `⊔` (Union): `Male ⊔ Female` - homme OU femme
- `¬` (Complement): `¬Male` - non-homme
- `∃R.C` (Exists restriction): `∃fatherOf.Human` - a un père humain
- `∀R.C` (Forall restriction): `∀childOf.Human` - tous les enfants sont humains

Ces opérateurs permettent de construire des ontologies complexes (ex: OWL en Web Sémantique).

##### TBox : Axiomes Terminologiques

La **TBox** (Terminological Box) definit les relations entre concepts :
- `Human`, `Male`, `Female`, `House`, `Father` - Concepts atomiques
- `fatherOf` - Role atomique
- Axiomes d'equivalence : `Female ≡ ¬Male`, `House ≡ ¬Human`

In [3]:
# --- 2.3.2 Definition des Concepts et TBox ---
if dl_imports_ok:
    # Concepts atomiques
    human = AtomicConcept("Human")
    male = AtomicConcept("Male")
    female = AtomicConcept("Female")
    house = AtomicConcept("House")
    father = AtomicConcept("Father")
    
    # Role atomique
    fatherOf = AtomicRole("fatherOf")
    
    # Individus
    bob = Individual("Bob")
    alice = Individual("Alice")
    
    # TBox : Axiomes Terminologiques
    femaleHuman = EquivalenceAxiom(female, human)
    maleHuman = EquivalenceAxiom(male, human)
    femaleNotMale = EquivalenceAxiom(female, Complement(male))
    maleNotFemale = EquivalenceAxiom(male, Complement(female))
    houseNotHuman = EquivalenceAxiom(house, Complement(human))
    
    print("TBox creee:")
    print(f"  Female = Human")
    print(f"  Male = Human")
    print(f"  Female = NOT Male")
    print(f"  Male = NOT Female")
    print(f"  House = NOT Human")
else:
    print("Imports DL non disponibles.")

TBox creee:
  Female = Human
  Male = Human
  Female = NOT Male
  Male = NOT Female
  House = NOT Human


### Analyse de la TBox

Les axiomes d'équivalence créés établissent une hiérarchie de concepts:

| Axiome | Signification | Impact sur le raisonnement |
|--------|---------------|----------------------------|
| `Female ≡ Human` | Les femmes sont humaines | `Alice : Female` ⇒ `Alice : Human` |
| `Male ≡ Human` | Les hommes sont humains | `Bob : Male` ⇒ `Bob : Human` |
| `Female ≡ ¬Male` | Femme = non-homme | Disjonction exclusive |
| `Male ≡ ¬Female` | Homme = non-femme | Symétrique du précédent |
| `House ≡ ¬Human` | Une maison n'est pas humaine | Domaines disjoints |

**Propriétés vérifiables:**
- **Consistency**: La TBox est cohérente (pas de contradictions)
- **Subsomption**: `Female ⊑ Human` est déductible
- **Disjointness**: `Male` et `Female` sont disjoints

##### ABox : Assertions sur les Individus

La **ABox** (Assertional Box) contient les faits sur les individus :
- `Alice : Human`, `Alice : Female`
- `Bob : Human`, `Bob : Male`
- `fatherOf(Bob, Alice)` - Bob est le pere d'Alice

In [4]:
# --- 2.3.3 ABox et Knowledge Base ---
if dl_imports_ok:
    # ABox : Assertions sur les individus
    aliceHuman = ConceptAssertion(alice, human)
    bobHuman = ConceptAssertion(bob, human)
    aliceFemale = ConceptAssertion(alice, female)
    bobMale = ConceptAssertion(bob, male)
    bobFatherOfAlice = RoleAssertion(bob, alice, fatherOf)
    
    # Construire la Knowledge Base
    dbs = DlBeliefSet()
    
    # TBox
    dbs.add(femaleHuman)
    dbs.add(maleHuman)
    dbs.add(femaleNotMale)
    dbs.add(maleNotFemale)
    dbs.add(houseNotHuman)
    
    # ABox
    dbs.add(aliceHuman)
    dbs.add(bobHuman)
    dbs.add(aliceFemale)
    dbs.add(bobMale)
    dbs.add(bobFatherOfAlice)
    
    print("Knowledge Base DL:")
    print(f"  TBox: {dbs.getTBox()}")
    print(f"  ABox: {dbs.getABox()}")
else:
    print("Imports DL non disponibles.")

Knowledge Base DL:
  TBox: [implies Female (not Male), implies House (not Human), implies Male Human, implies Female Human, implies Male (not Female)]
  ABox: [instance Bob Male, instance Alice Human, instance Bob Human, related Bob Alice fatherOf, instance Alice Female]


### Interpretation de la Knowledge Base DL

**TBox (5 axiomes):**
- 2 axiomes de subsomption: `Male ⊑ Human`, `Female ⊑ Human`
- 2 axiomes de disjonction: `Female ≡ ¬Male`, `Male ≡ ¬Female`
- 1 axiome de domaine: `House ≡ ¬Human`

**ABox (5 assertions):**
- 2 assertions de concept: `Alice : Human`, `Bob : Human`
- 2 assertions de genre: `Alice : Female`, `Bob : Male`
- 1 assertion de rôle: `fatherOf(Bob, Alice)`

**Cohérence de la KB:**
La base de connaissances est cohérente car:
1. Alice et Bob respectent les contraintes de disjonction (`Female` ≠ `Male`)
2. Les assertions de rôle (`fatherOf`) sont compatibles avec les concepts
3. Aucune contradiction n'apparaît entre TBox et ABox

> **Applications réelles:** Les ontologies biomédicales (SNOMED CT), les taxonomies scientifiques, et le Web Sémantique utilisent des structures similaires.

##### Raisonnement DL

Le `NaiveDlReasoner` permet de repondre a des requetes sur la KB :
- **Subsomption** : `Female ⊑ Human` ?
- **Instance** : `Tweety : Human` ?
- **Negation** : `Alice : Male` ?

In [5]:
# --- 2.3.4 Raisonnement DL ---
if dl_imports_ok:
    # Preparer une KB pour le raisonnement
    dbs_reason = DlBeliefSet()
    tweety = Individual("Tweety")
    tweetyMale = ConceptAssertion(tweety, male)
    tweetyHuman = ConceptAssertion(tweety, human)
    
    dbs_reason.add(aliceFemale)
    dbs_reason.add(tweetyMale)
    dbs_reason.add(maleNotFemale)
    dbs_reason.add(aliceHuman)
    dbs_reason.add(femaleHuman)
    
    reasoner_dl = NaiveDlReasoner()
    
    print("Requetes DL:")
    
    # Query 1: Female implique Human ?
    q1_dl = femaleHuman
    print(f"  Female = Human ? {reasoner_dl.query(dbs_reason, q1_dl)}")
    
    # Query 2: Tweety est Humain ?
    dbs_reason.add(maleHuman)
    q2_dl = tweetyHuman
    print(f"  Tweety : Human ? {reasoner_dl.query(dbs_reason, q2_dl)}")
    
    # Query 3: Alice est Male ?
    q3_dl = ConceptAssertion(alice, male)
    print(f"  Alice : Male ? {reasoner_dl.query(dbs_reason, q3_dl)}")
else:
    print("Imports DL non disponibles.")

Requetes DL:
  Female = Human ? True
  Tweety : Human ? True
  Alice : Male ? False


### Analyse des Resultats de Raisonnement DL

| Requête | Résultat | Explication |
|---------|----------|-------------|
| `Female = Human ?` | **True** | L'axiome `Female ≡ Human` est dans la KB |
| `Tweety : Human ?` | **True** | Par transitivité: `Tweety : Male` + `Male ⊑ Human` |
| `Alice : Male ?` | **False** | Contradiction avec `Alice : Female` et `Female ≡ ¬Male` |

**Mécanismes de raisonnement utilisés:**
1. **Subsumption checking**: Vérifier si un concept en subsume un autre
2. **Instance checking**: Déterminer si un individu appartient à un concept
3. **Consistency checking**: Détecter les contradictions (ex: `Alice : Male` et `Alice : Female`)

**Limitations du `NaiveDlReasoner`:**
- Algorithme exhaustif (force brute)
- Complexité exponentielle pour les ontologies complexes
- Pour des ontologies réelles, préférer des raisonneurs optimisés:
  - **Pellet**: Supporte OWL 2 DL
  - **HermiT**: Raisonneur OWL basé sur les tableaux
  - **ELK**: Optimisé pour le profil EL++ (polynomial)

### 2.4 Logique Modale (ML)
<a id="2.4"></a>

La logique modale ajoute des operateurs pour qualifier les propositions :

| Operateur | Symbole Tweety | Signification |
|-----------|----------------|---------------|
| **Necessite** | `[]` (Box) | "Necessairement vrai dans tous les mondes" |
| **Possibilite** | `<>` (Diamond) | "Possiblement vrai dans au moins un monde" |

**Exemple :** `[](p => q)` signifie "Il est necessaire que si p alors q".

**Semantique de Kripke :** Les formules modales sont evaluees par rapport a des **mondes possibles** relies par une relation d'accessibilite.

**Classes Tweety :**
- `MlBeliefSet` : Base de croyances modales
- `MlParser` : Parseur de formules modales
- `SPASSMlReasoner` : Raisonneur utilisant SPASS (recommande)
- `SimpleMlReasoner` : Raisonneur naif (peut bloquer)

In [6]:
# --- 2.4.1 Imports ML ---
print("--- 2.4.1 Logique Modale : Imports ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
    ml_imports_ok = False
else:
    ml_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        
        from org.tweetyproject.logics.ml.syntax import MlBeliefSet
        from org.tweetyproject.logics.ml.parser import MlParser
        from org.tweetyproject.logics.fol.syntax import FolSignature, FolFormula
        from org.tweetyproject.logics.commons.syntax import Predicate
        from org.tweetyproject.logics.ml.reasoner import AbstractMlReasoner, SimpleMlReasoner, SPASSMlReasoner
        
        FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
        print("Imports ML reussis.")
        ml_imports_ok = True
        
    except ImportError as e:
        print(f"ERREUR d'import ML : {e}")
    except Exception as e:
        print(f"ERREUR : {e}")

--- 2.4.1 Logique Modale : Imports ---
Imports ML reussis.


### Transition : De la Description Logic a la Logique Modale

Nous passons maintenant d'une logique **taxonomique** (DL) à une logique **temporelle/épistémique** (ML):

| Aspect | Description Logic (DL) | Logique Modale (ML) |
|--------|------------------------|---------------------|
| **Focus** | Classification de concepts | Modalités (nécessité, possibilité) |
| **Monde** | Un seul monde (ontologie) | Mondes possibles multiples |
| **Opérateurs** | `⊓`, `⊔`, `¬`, `∃`, `∀` | `[]` (nécessité), `<>` (possibilité) |
| **Sémantique** | Interprétation ensembliste | Sémantique de Kripke |
| **Applications** | Web sémantique, ontologies | Raisonnement temporel, épistémique |

**Exemple de correspondance:**
- DL: `∀fatherOf.Human` → "Tous ceux dont on est père sont humains"
- ML: `[](p => q)` → "Nécessairement, si p alors q (dans tous les mondes)"

La logique modale ajoute une dimension **intensionnelle** absente de DL classique.

##### Signature et Parsing Modal

Syntaxe des formules modales Tweety :
- `[]` : Box (necessite)
- `<>` : Diamond (possibilite)  
- Combinable avec connecteurs classiques : `&&`, `||`, `!`, `=>`

In [7]:
# --- 2.4.2 Signature et Parsing ML ---
if ml_imports_ok:
    # Signature avec predicats 0-aires (propositions)
    sig_ml = FolSignature()
    p = Predicate("p", 0)
    q = Predicate("q", 0)
    r = Predicate("r", 0)
    sig_ml.add(p)
    sig_ml.add(q)
    sig_ml.add(r)
    
    parser_ml = MlParser()
    parser_ml.setSignature(sig_ml)
    kb_ml = MlBeliefSet()
    
    # Formules modales
    formulas_ml = [
        "!(<>(p))",             # NOT possible p
        "p || r",               # p OR r
        "!r || [](q && r)",     # NOT r OR necessarily(q AND r)
        "[](r && <>(p || q))",  # necessarily(r AND possibly(p OR q))
        "!p && !q"              # NOT p AND NOT q
    ]
    
    print("Parsing des formules modales:")
    for f_str in formulas_ml:
        try:
            kb_ml.add(parser_ml.parseFormula(f_str))
            print(f"  [OK] {f_str}")
        except Exception as e:
            print(f"  [ERREUR] {f_str}: {e}")
    
    print(f"\nKB Modale: {kb_ml.size()} formules")
else:
    print("Imports ML non disponibles.")

Parsing des formules modales:
  [OK] !(<>(p))
  [OK] p || r
  [OK] !r || [](q && r)
  [OK] [](r && <>(p || q))
  [OK] !p && !q

KB Modale: 5 formules


### Analyse des Formules Modales Parsees

| Formule | Traduction naturelle | Type |
|---------|----------------------|------|
| `!(<>(p))` | "Il n'est PAS possible que p" ≡ `[](!p)` | Nécessité (par dualité) |
| `p \|\| r` | "p OU r" (formule propositionnelle) | Classique |
| `!r \|\| [](q && r)` | "NON r OU nécessairement (q ET r)" | Mixte (classique + modal) |
| `[](r && <>(p \|\| q))` | "Nécessairement (r ET possiblement (p OU q))" | Imbrication modale |
| `!p && !q` | "NON p ET NON q" | Classique |

**Axiomes et dualités modales:**
- **Dualité Box-Diamond**: `[]φ ≡ ¬<>¬φ` et `<>φ ≡ ¬[]¬φ`
- **Axiome K**: `[](p => q) => ([]p => []q)` (distribution)
- **Axiome T**: `[]p => p` (réflexivité)
- **Axiome 4**: `[]p => [][]p` (transitivité)

> **Note:** Le système modal de Tweety supporte S5 (système le plus fort) qui inclut tous ces axiomes.

##### Raisonnement Modal avec SPASS

Le raisonnement modal necessite un prouveur externe comme **SPASS**.

> **Note:** `SimpleMlReasoner` peut bloquer indefiniment car il enumere tous les mondes possibles.

In [8]:
# --- 2.4.3 Raisonnement ML avec SPASS ---
if ml_imports_ok and kb_ml.size() > 0:
    spass_path = get_tool_path('SPASS') if 'get_tool_path' in globals() else None
    
    if spass_path:
        print(f"SPASS detecte: {spass_path}")
        try:
            spass_reasoner = SPASSMlReasoner(JString(spass_path))
            AbstractMlReasoner.setDefaultReasoner(spass_reasoner)
            ml_reasoner = AbstractMlReasoner.getDefaultReasoner()
            print(f"Raisonneur: {ml_reasoner.getClass().getSimpleName()}")
            
            # Test une seule requete d'abord pour verifier si SPASS fonctionne
            queries_ml = ["[](!p)", "<>(q || r)", "p", "r", "[](q)"]
            test_query = parser_ml.parseFormula(queries_ml[0])
            spass_works = False
            
            try:
                result = ml_reasoner.query(kb_ml, JObject(test_query, FolFormula_class))
                spass_works = True
            except Exception as e_test:
                error_str = str(e_test)
                if "error=740" in error_str or "elevation" in error_str.lower():
                    print("\n⚠️ ERREUR PERMISSIONS WINDOWS (error=740)")
                    print("   SPASS.exe requiert des privileges administrateur.")
                    print("\n   Solutions:")
                    print("   1. Executer Jupyter en tant qu'administrateur")
                    print("   2. Supprimer le fichier et reinstaller SPASS depuis:")
                    print("      https://www.spass-prover.org/download/binaries/")
                    print("      (choisir la version sans installeur si disponible)")
                    print("   3. Clic droit sur SPASS.exe > Proprietes > Compatibilite")
                    print("      -> Decocher 'Executer en tant qu'administrateur'")
                    print("\n   Le parsing ML fonctionne, seul le raisonnement necessite SPASS.")
                else:
                    print(f"\n   Erreur SPASS inattendue: {e_test}")
            
            # Si SPASS fonctionne, executer toutes les requetes
            if spass_works:
                print("\nRequetes modales:")
                for q_str in queries_ml:
                    try:
                        query = parser_ml.parseFormula(q_str)
                        result = ml_reasoner.query(kb_ml, JObject(query, FolFormula_class))
                        print(f"  {q_str} : {result}")
                    except Exception as e:
                        print(f"  {q_str} : ERREUR - {e}")
                    
        except Exception as e:
            print(f"Erreur configuration SPASS: {e}")
    else:
        print("SPASS non configure.")
        print("  SimpleMlReasoner bloque indefiniment (enumeration exhaustive).")
        print("  Installez SPASS: https://www.spass-prover.org/")
else:
    print("KB modale vide ou imports manquants.")

SPASS detecte: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\ext_tools\spass\SPASS.exe
Raisonneur: SPASSMlReasoner

⚠️ ERREUR PERMISSIONS WINDOWS (error=740)
   SPASS.exe requiert des privileges administrateur.

   Solutions:
   1. Executer Jupyter en tant qu'administrateur
   2. Supprimer le fichier et reinstaller SPASS depuis:
      https://www.spass-prover.org/download/binaries/
      (choisir la version sans installeur si disponible)
   3. Clic droit sur SPASS.exe > Proprietes > Compatibilite
      -> Decocher 'Executer en tant qu'administrateur'

   Le parsing ML fonctionne, seul le raisonnement necessite SPASS.


### Diagnostic : Erreur SPASS et Solutions

**Problème identifié:** Erreur Windows 740 (élévation de privilèges requise)

**Pourquoi SPASS nécessite-t-il des privilèges admin?**
- L'exécutable a le flag "requireAdministrator" dans son manifeste
- Problème courant avec les anciens binaires Windows

**Solutions par ordre de préférence:**

| Solution | Avantages | Inconvénients |
|----------|-----------|---------------|
| **1. Désactiver le flag admin** | Permanent, pas de droits admin requis | Nécessite modification des propriétés |
| **2. Jupyter en admin** | Solution rapide | Tous les notebooks ont des droits élevés (risque) |
| **3. Version portable SPASS** | Pas de problème de privilèges | Nécessite recompilation ou version alternative |
| **4. Utiliser un autre prouveur** | Alternative viable | Nécessite configuration différente |

**Alternatives à SPASS pour la logique modale:**
- **MleanCoP**: Prouveur modal basé sur connexion
- **MleanTAP**: Tableaux modaux
- **MSPASS**: Version modernisée de SPASS (si disponible)

> **Pour ce notebook:** Le parsing modal fonctionne parfaitement. Seul le raisonnement automatique nécessite SPASS. Les requêtes peuvent être vérifiées manuellement avec la sémantique de Kripke.

### 2.5 Autres Logiques (Apercu QBF, CL)
<a id="2.5"></a>

Tweety supporte d'autres logiques que nous survolons ici :

**QBF (Quantified Boolean Formulas) :**
Extension de SAT avec quantificateurs sur les variables booleennes.
- `forall X: exists Y: (X || Y)`
- Complexite : PSPACE-complet

**Logique Conditionnelle (CL) :**
Extension de la logique propositionnelle avec l'operateur conditionnel `|~` (normalement implique).
- `bird |~ flies` : "Normalement, les oiseaux volent"
- Gere les exceptions et le raisonnement non-monotone

In [9]:
# --- 2.5.1 Imports QBF ---
print("--- 2.5.1 QBF (Quantified Boolean Formulas) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
    qbf_imports_ok = False
else:
    qbf_imports_ok = False
    try:
        from org.tweetyproject.logics.qbf.syntax import (
            ExistsQuantifiedFormula, ForallQuantifiedFormula
        )
        from org.tweetyproject.logics.pl.syntax import Proposition, PlBeliefSet, Disjunction
        from org.tweetyproject.logics.qbf.reasoner import QbfSolver
        from java.util import ArrayList
        
        print("Imports QBF reussis.")
        qbf_imports_ok = True
        
    except ImportError as e:
        print(f"ERREUR d'import QBF : {e}")
    except Exception as e:
        print(f"ERREUR : {e}")

--- 2.5.1 QBF (Quantified Boolean Formulas) ---
Imports QBF reussis.


### Introduction a QBF : Au-dela de SAT

**Hierarchie de complexite:**

| Logique | Complexité | Quantificateurs | Exemple |
|---------|------------|-----------------|---------|
| **SAT** | NP-complet | Aucun (variables libres) | `(x ∨ y) ∧ (¬x ∨ z)` |
| **QBF** | PSPACE-complet | ∀, ∃ sur variables booléennes | `∀x ∃y: (x ⇔ y)` |
| **FOL** | Indécidable | ∀, ∃ sur domaines infinis | `∀x ∃y: P(x,y)` |

**Puissance expressive de QBF:**
- Peut encoder des problèmes de planification avec incertitude
- Modélise les jeux à deux joueurs (∀ = adversaire, ∃ = nous)
- Résout des problèmes de vérification formelle (model checking)

**Exemple concret:**
```
∀x ∃y: (x => y)
```
"Pour toute valeur de x, il existe y tel que (x implique y)"
- Si x=0, choisir y=0 ou y=1 (0=>0 et 0=>1 sont vrais)
- Si x=1, choisir y=1 (1=>1 est vrai)
- Formule **satisfiable** (stratégie gagnante : y=1 toujours)

**Solveurs QBF modernes:**
- **QuAbS**: Basé sur la recherche DPLL
- **CAQE**: Expansion clausale
- **DepQBF**: Résolution Q

##### Construction de formules QBF

Une formule QBF combine quantificateurs et formules propositionnelles :
- `exists X: (X || Y)` - Il existe une valeur de X telle que X ou Y
- `forall X: exists Y: (X <=> Y)` - Formules imbriquees

In [10]:
# --- 2.5.2 Construction de formules QBF ---
if qbf_imports_ok:
    from java.util import HashSet  # IMPORTANT: QBF attend un Set, pas ArrayList!
    
    # Variables propositionnelles
    x = Proposition("x")
    y = Proposition("y")
    
    # Formule: x OR y
    vars_disj = ArrayList()
    vars_disj.add(x)
    vars_disj.add(y)
    inner_formula = Disjunction(vars_disj)
    print(f"Formule de base: {inner_formula}")
    
    # exists x: (x OR y) - utiliser HashSet car le constructeur attend un Set
    exists_x_vars = HashSet()
    exists_x_vars.add(x)
    exists_formula = ExistsQuantifiedFormula(inner_formula, exists_x_vars)
    print(f"Formule QBF: {exists_formula}")
    
    # forall y: exists x: (x OR y) - meme correction avec HashSet
    forall_y_vars = HashSet()
    forall_y_vars.add(y)
    forall_exists_formula = ForallQuantifiedFormula(exists_formula, forall_y_vars)
    print(f"Formule QBF imbriquee: {forall_exists_formula}")
    
    # Alternative: utiliser le constructeur simplifie (Proposition unique)
    print("\n--- Variante avec constructeur simplifie ---")
    # exists z: (z) - une seule variable
    z = Proposition("z")
    exists_z = ExistsQuantifiedFormula(z, z)  # (formula, single_proposition)
    print(f"exists z: z = {exists_z}")
else:
    print("Imports QBF non disponibles.")

Formule de base: x||y
Formule QBF: exists x: (x||y)
Formule QBF imbriquee: forall y: (exists x: (x||y))

--- Variante avec constructeur simplifie ---
exists z: z = exists z: (z)


### Analyse des Formules QBF Construites

**Formule 1 : `exists x: (x||y)`**
- **Sens:** "Il existe x tel que (x OU y)"
- **Satisfiabilité:** Toujours vraie (choisir x=1, alors x||y=1 quelle que soit y)
- **Stratégie:** x=1 est une stratégie gagnante

**Formule 2 : `forall y: exists x: (x||y)`**
- **Sens:** "Pour tout y, il existe x tel que (x OU y)"
- **Satisfiabilité:** Toujours vraie
- **Stratégie:** Fonction de Skolem f(y) = 1 (choisir x=1 indépendamment de y)

**Ordre des quantificateurs:**
L'ordre est **crucial** en QBF:

| Formule | Résultat | Raison |
|---------|----------|--------|
| `∃x ∀y: (x ⇔ y)` | **UNSAT** | x doit être égal à tous les y (impossible) |
| `∀y ∃x: (x ⇔ y)` | **SAT** | Pour chaque y, choisir x=y |

**Construction avec HashSet vs ArrayList:**
Tweety utilise `Set<Proposition>` pour éviter les doublons de variables quantifiées.
- `HashSet`: Pas d'ordre, pas de doublons ✓
- `ArrayList`: Ordre préservé, doublons possibles ✗

> **Astuce:** Pour des formules QBF complexes, préférer le parsing textuel au lieu de la construction programmatique.

## Logique Conditionnelle (CL)

La logique conditionnelle permet d'exprimer des regles avec exceptions :
- `bird |~ flies` : "Normalement, les oiseaux volent"
- `penguin |~ !flies` : "Les pingouins ne volent pas (exception)"

In [11]:
# --- 2.5.3 Logique Conditionnelle ---
print("--- 2.5.3 Logique Conditionnelle (CL) ---")

if jvm_ready:
    cl_imports_ok = False
    try:
        from org.tweetyproject.logics.cl.syntax import Conditional, ClBeliefSet
        from org.tweetyproject.logics.pl.syntax import Proposition, Negation
        from org.tweetyproject.logics.cl.reasoner import SimpleCReasoner
        
        print("Imports CL reussis.")
        cl_imports_ok = True
        
        # Propositions
        bird = Proposition("bird")
        flies = Proposition("flies")
        penguin = Proposition("penguin")
        
        # Conditionnels
        bird_flies = Conditional(flies, bird)          # bird |~ flies
        penguin_not_flies = Conditional(Negation(flies), penguin)  # penguin |~ !flies
        
        print(f"\nConditionnel 1: {bird_flies}")
        print(f"Conditionnel 2: {penguin_not_flies}")
        
        # Base de croyances conditionnelle
        cl_kb = ClBeliefSet()
        cl_kb.add(bird_flies)
        cl_kb.add(penguin_not_flies)
        print(f"\nKB Conditionnelle: {cl_kb}")
        
    except ImportError as e:
        print(f"ERREUR d'import CL : {e}")
    except Exception as e:
        print(f"ERREUR : {e}")
else:
    print("JVM non demarree.")

--- 2.5.3 Logique Conditionnelle (CL) ---
Imports CL reussis.

Conditionnel 1: (bird|flies)
Conditionnel 2: (penguin|!flies)

KB Conditionnelle: { (penguin|!flies), (bird|flies) }


### Interpretation de la Logique Conditionnelle

**Conditionnels crees:**

| Conditionnel | Notation CL | Signification |
|--------------|-------------|---------------|
| `(bird\|flies)` | `bird \|~ flies` | "Normalement, les oiseaux volent" |
| `(penguin\|!flies)` | `penguin \|~ ¬flies` | "Normalement, les pingouins ne volent pas" |

**Difference avec l'implication classique:**

| Logique | Operateur | Comportement avec exceptions |
|---------|-----------|------------------------------|
| **Classique** | `bird => flies` | Si `penguin ⊑ bird` et `penguin => ¬flies`, KB incohérente |
| **Conditionnelle** | `bird \|~ flies` | Les exceptions (pingouins) n'invalident pas la règle générale |

**Semantique de la logique conditionnelle:**
- **Modeles preferentiels:** Les mondes "normaux" sont préférés
- **Ordres sur les mondes:** Classement par typicalité (mondes avec oiseaux volants > mondes avec pingouins)
- **Raisonnement non-monotone:** Ajouter `penguin(Tweety)` peut invalider `flies(Tweety)` précédemment inféré

**Applications:**
- **Raisonnement par défaut:** "Les adultes ont un emploi" (sauf retraités, étudiants)
- **Diagnostic médical:** "Fièvre implique infection" (sauf causes non-infectieuses)
- **Droit:** "Les contrats sont valides" (sauf vices de consentement)

> **Raisonneurs CL:** System Z, System P, rational closure (implémentés dans Tweety via `SimpleCReasoner`).

---

## Resume

Ce notebook a couvert:
- **Description Logic (DL)**: Concepts, roles, ABox/TBox, classification
- **Logique Modale (ML)**: Operateurs Box/Diamond, semantique de Kripke, SPASS
- **QBF**: Formules booleennes quantifiees, complexite PSPACE
- **Logique Conditionnelle (CL)**: Regles avec exceptions, raisonnement non-monotone

**Points cles:**
- Chaque logique etend la logique propositionnelle differemment
- Les raisonneurs externes (SPASS, EProver) sont souvent necessaires
- Tweety fournit des parseurs et structures pour toutes ces logiques

## Prochaines etapes

Le notebook suivant explore la **revision de croyances** et la gestion de l'incoherence.

---

**Navigation**: [Tweety-2-Basic-Logics](Tweety-2-Basic-Logics.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb)